# Peak propagation

In [2]:
from DLC_for_WBFM.utils.projects.finished_project_data import finished_project_data
import numpy as np
import matplotlib.pyplot as plt
import pickle
import napari, zarr
from skimage.transform import PiecewiseAffineTransform, warp
from scipy.spatial import KDTree

In [3]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
dat = finished_project_data.load_final_project_data_from_config(fname)

In [5]:
fname = r"C:\dlc_stacks\Test_project\2-training_data\raw\match_dat.pickle"
with open(fname, 'rb') as f:
    matches = pickle.load(f)
    
fname = r"C:\dlc_stacks\Test_project\2-training_data\raw\frame_dat.pickle"
with open(fname, 'rb') as f:
    frames = pickle.load(f)

C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator GaussianProcessRegressor from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# Check default affine to the points

In [11]:
pair = (0, 1)
f0, f1 = frames[pair[0]], frames[pair[1]]

In [17]:
# Get initial affine locations
affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Get image
vol = dat.red_data[pair[1], ...]
# vol_filtered = gaussian(vol, sigma=1)

# Get locations of target neurons
target_zxy = f1.neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

In [18]:
# Only check the ones with no match
f0_to_f1_matches = matches[pair].get_f0_to_f1_dict()
has_match_ind = list(f0_to_f1_matches.keys())
no_match_ind = list(range(frames[pair[0]].num_neurons()))
[no_match_ind.remove(i) for i in has_match_ind]

# Set up kdtree to check nearest neighbor
tree = KDTree(target_zxy)

# Parameters
brightness_thresh = 25
distance_thresh = 5.0

In [21]:
# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_zxy_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_zxy_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

affine_zxy_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_zxy_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [30]:
new_f1_zxy = []
new_matches = []
next_f1_ind = frames[pair[1]].num_neurons() + 1

print("Added? Brightness? Neighbor?")

for ind_f0 in no_match_ind:
    pushed_zxy = affine_zxy[ind_f0, :]
    z, x, y = int(pushed_zxy[0]), int(pushed_zxy[1]), int(pushed_zxy[2])
    brightness = vol[z, x, y]
    
    nn_dist, _ = tree.query(pushed_zxy, k=1)
    
    to_keep = (brightness > brightness_thresh) and (nn_dist > distance_thresh)
    print(to_keep, brightness, nn_dist)
    if to_keep:
        new_matches.append([ind_f0, next_f1_ind, 1.0])
        next_f1_ind += 1
        new_f1_zxy.append(pushed_zxy)

print(f"Added {len(new_matches)}/{len(no_match_ind)} neurons from f0 to f1")

Added? Brightness? Neighbor?
True 45 6.821041040113981
False 0 269.7365020135102
False 0 269.7365020135102
False 0 269.7365020135102
True 26 13.320621685171922
False 41 4.8755225475571695
True 52 6.286293802576122
True 33 5.373181147692626
True 39 6.086906786119055
False 21 14.908471174282477
Added 5/10 neurons from f0 to f1


In [24]:
# Plot using napari
v = napari.view_image(dat.red_data[pair[1]], ndisplay=3)
v.add_points(new_f1_zxy, size=3, face_color='green', symbol='x', n_dimensional=True)
# v.add_points(affine_zxy_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
v.add_points(target_zxy_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True)
v.add_points(target_zxy_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True)

<Points layer 'Points [2]' at 0x1f18c682f88>

# Improve the affine model using additional ORB feature matches

In [25]:
matches[pair]

FramePair with 115 matches 

In [26]:
f0

ReferenceFrame with 125 neurons 

In [27]:
f1

ReferenceFrame with 136 neurons 